# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.options.mode.chained_assignment = None

Шаг 1. Загрузим датафреймы, изучим информацию о них, проверим их на дубликаты.

In [2]:
df0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
df1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
df2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

df0.info()
df1.info()
df2.info()
df0.duplicated().value_counts()
df1.duplicated().value_counts()
df2.duplicated().value_counts()

df0 = df0.drop(['id'], axis='columns')
df1 = df1.drop(['id'], axis='columns')
df2 = df2.drop(['id'], axis='columns')
display(df0.head())
display(df1.head())
display(df2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


Типы данных в столбцах соответствуют природе этих данных, преобразования не требуются. Дубликатов нет.
Удалим столбец 'id' с типом данных object, так как модели не воспринимают буквы, других проблем с предподготовкой данных не вижу.

## Обучение и проверка модели

Создадим функцию model для того, чтобы сразу в каждом датафрейме выделить целевой признак и разбить датафреймы на тренировочную и валидационную выборки. Внутри функции обучим модель, подсчитаем MSE, RMSE и предсказание среднего объема скважин по региону.

In [3]:
def model(df):
    features = df.drop(['product'], axis = 1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])

    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    df['predicted'] = model.predict(features) #добавим столбец с предсказаниями модели прямо в датафрейм, чтобы в дальнейших функциях не путаться с индексами
    mean = predictions.mean()
    mse = mean_squared_error(target_valid, predictions)
    print("MSE =", mse)
    print("RMSE =", mse ** 0.5)
    print("Predicted_mean =", predictions.mean())
    return mean, predictions

In [4]:
model(df0) 
df0.head()

MSE = 1412.2129364399243
RMSE = 37.5794217150813
Predicted_mean = 92.59256778438035


,f0,f1,f2,product,predicted
0,0.705745,-0.497823,1.221170,105.280062,124.553920
1,1.334711,-0.340164,4.365080,73.037750,192.763405
2,1.022732,0.151990,1.419926,85.265647,125.186291
3,-0.032172,0.139033,2.978566,168.620776,155.368884
4,1.988431,0.155413,4.751769,154.036647,199.573411


In [5]:
model(df1)
df1.head()

MSE = 0.7976263360391157
RMSE = 0.893099286775617
Predicted_mean = 68.728546895446


,f0,f1,f2,product,predicted
0,-15.001348,-8.276000,-0.005876,3.179103,89.017978
1,14.272088,-3.475083,0.999183,26.953261,96.553612
2,6.263187,-5.948386,5.001160,134.766305,290.875231
3,-13.081196,-11.506057,4.999415,137.945408,316.560503
4,12.702195,-8.147433,5.004363,134.766305,282.903146


In [6]:
model(df2)
df2.head()

MSE = 1602.3775813236196
RMSE = 40.02970873393434
Predicted_mean = 94.96504596800489


,f0,f1,f2,product,predicted
0,-1.146987,0.963328,-0.828965,27.758673,78.477490
1,0.262778,0.269839,-2.530187,56.069697,44.885688
2,0.194587,0.289035,-5.586433,62.871910,-15.693094
3,2.236060,-0.553760,0.930038,114.572842,113.631571
4,-0.515993,1.716266,5.899011,149.600746,211.802858


Вывод. Точнее всего предсказания получаются в регионе 1 (RMSE меньше единицы), но и прогноз по объему скважин там существенно ниже соседних регионов (69 против 90+).

## Подготовка к расчёту прибыли

Бюджет на разработку скважин в регионе — 10 млрд рублей.

Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

Для разработки выбирают 200 точек.

In [7]:
BUDGET = 10000000000 
REVENUE = 450000 
PROD_ONE_WELL = BUDGET/200/REVENUE
PROD_ONE_WELL

111.11111111111111

Для безубыточности производства скважина должна содержать 111 тысяч баррелей нефти. Ниже сравним это со средними данными по регионам.

In [8]:
print(df0['product'].mean())
print(df1['product'].mean())
print(df2['product'].mean())


92.50000000000001
68.82500000000002
95.00000000000004


Хочется сделать 2 вывода. 
1) Модели выше выдают неплохое качество по среднему значению запасов нефти.

2) Средние значения по регионам значительно уступают целевому значению безубыточности в 111 тысячу баррелей.

## Расчёт прибыли и рисков 

Преобразуем предсказания моделей в датафреймы пандас, чтобы их можно было сортировать по величине запасов. 

In [9]:
def profit(df):
    sort_predicted = df.sort_values('predicted', ascending = False).head(200)
    sort_real = df[df.index.isin(sort_predicted.index)]
    revenues = sort_real['product'].sum()*REVENUE - BUDGET
    return revenues

print('Предсказанная прибыль в регионе 0:', profit(df0))
print('Предсказанная прибыль в регионе 1:', profit(df1))
print('Предсказанная прибыль в регионе 2:', profit(df2))

Предсказанная прибыль в регионе 0: 2948960411.773096
Предсказанная прибыль в регионе 1: 2415086696.681511
Предсказанная прибыль в регионе 2: 2555578577.505953


<div class="alert alert-block alert-info">
<b>Совет: </b> Эту функцию лучше было бы назвать profit, так как revenue – это выручка.
</div>

In [10]:
def risk_profit(df):
    state = np.random.RandomState(12345)
    target = df['product']
    predictions = df['predicted']
    values = []
    for i in range(1000):
        predictions_subsample = predictions.sample(n=500, replace=True, random_state=state)
        data = df[df.index.isin(predictions_subsample.index)]
        value = profit(data)
        values.append(value)
    
    values = pd.Series(values)    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
    loss_risk = values[values < 0].count() / values.count()
    losses = (values < 0).mean()
                 
    return(print(f'95%й доверительный интервал {lower} - {upper} руб.'),
           print("Средняя прибыль по региону:", mean),
           print(losses),
           print("Вероятность убытков по региону:", f'{loss_risk*100}%'))
    

In [11]:
risk_profit(df0)

95%й доверительный интервал -201086434.5541483 - 900656314.1569986 руб.
Средняя прибыль по региону: 333962035.55452675
0.097
Вероятность убытков по региону: 9.700000000000001%


(None, None, None, None)

In [12]:
risk_profit(df1)

95%й доверительный интервал 33153416.18845363 - 813156428.839896 руб.
Средняя прибыль по региону: 441614931.09542817
0.014
Вероятность убытков по региону: 1.4000000000000001%


(None, None, None, None)

In [13]:
risk_profit(df2)

95%й доверительный интервал -182019253.3325233 - 889544958.2930584 руб.
Средняя прибыль по региону: 370377844.7408863
0.09
Вероятность убытков по региону: 9.0%


(None, None, None, None)

Условия проекта гласят, что после оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Данному условию удовлетворяет лишь регион 1. Помимо минимального риска убытков, средняя прибыль в этом регионе также превышает соседние. Выбираем его.

### Общие выводы
В данном проекте мы изучили нефтяные месторождения трех регионов с целью выбора самого выгодного для бурения скважин в рамках обозначенного бюджета. Для этого мы:
* изучили три датасета, 
* обучили по ним модель линейной регрессии, 
* проанализировали бизнес-термины, составили функции для расчета прибыли с учетом различных условий бизнеса;
* с помощью процедуры бутстреп оценили соотношение рисков и прибыли, что позволило выбрать оптимальный регион для бурения новых скважин. 

Отметим, что по первоначальным данным обучения моделей этот регион казался наименее интересным с точки зрения запасов нефти, однако бутстрепирование показало, что именно он наименее рискованный и обещает самую стабильную и высокую прибыль для бизнеса.